# Watervraag, -aanbod en -tekort 
Dit script schaalt Mozart-uitvoer van LSW-schaal naar peilgebieden-schaal. 
Ook wordt door middel van de basisprognoses een factor toegekend om zo de verandering van de watervraag, -aanbod en -tekort in de toekomst in beeld te brengen.
Tot slot wordt een Monte Carlo analyse uitgevoerd waarmee de nauwkeurigheid van de resultaten beoordeeld wordt.
De aanpak wordt schematisch weergegeven in onderstaand figuur:

<img src="flowchart_hdsr.png">

# Data
* Topografische basisbestand : top10nl
* Uitvoer Mozart: Validatie LHM 4.1 (periode 2011-2020)
* Peilgebieden: data Hoogheemraadschap De Stichtse Rijnlanden
* Toekomstscenarios: Basisprognoses DPZW

1: Importeren van packages

In [ ]:
# generieke imports
import copy
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import matplotlib.pyplot as plt

# specifieke imports
import functies_watervraag_aanbod as f_wa


2: Definiëren  van bestandsnamen

In [ ]:
# definitie van een dictionary voor de bestandsnamen
fNames = dict()

# top10nl
fNames['top10nl'] = r'..\data\top10nl\clip\top10nl_terrein_water_weg.gpkg'

# koppelcodes top10nl en Mozart
fNames['top10nl_koppeling'] = r'..\data\top10nl\koppeling_top10nl_mozart.xlsx'

# LSW-polygonen
fNames['lsw'] = r'..\data\geo\lsw\lsw_hdsr_large.shp'

# peilgebieden
fNames['peilgebieden'] = r'..\data\GISdata\peilgebieden.gpkg'

# uitvoer mozart
fNames['mozart_out'] = r'..\data\mozart\lswwaterbalans.out'


3: Inlezen van de invoerdata

In [ ]:
data = dict()
data['top10nl'] = gpd.read_file(fNames['top10nl'])
data['lsw'] = gpd.read_file(fNames['lsw'])
data['peilgebieden'] = gpd.read_file(fNames['peilgebieden'])

# definieer RD-projectie voor LSW's
data['lsw'] = data['lsw'].set_crs(epsg=28992)

# lees het grote tekstbestand met de uitvoer van Mozart
df_mozart = f_wa.inlezen_mozart_data(fNames['mozart_out'],
                                schrijf_csv=False)

# verkrijg kolomnaam van vraag- en aanbodtermen
demand_columns, alloca_columns = f_wa.kolomnamen_vraag_aanbod(df_mozart)

#%% inlezen koppelcodes
data['koppelcodes'] = pd.read_excel(fNames['top10nl_koppeling'])

# voeg koppelcodes toe aan GeoDataFrame van top10nl
data['top10nl_gekoppeld'] = data['top10nl'].merge(data['koppelcodes'], on='typelandgebruik')

print('Data correct ingeladen.')

4: schaal vervolgens de mozart-uitvoer per LSW naar peilgebiedschaal

In [ ]:
# verkrijg de LSW-nummers
LSW_NRS = data['lsw']['LSWFINAL']

# definieer dictionary voor resultaten per peilgebied
dict_peilgebieden = dict()

# loop over elke LSW
for _, lsw_nr in tqdm(LSW_NRS[:].iteritems()):
    # print(lsw_nr)

    # selecteer LSW-gegevens
    lsw_data = data['lsw'][data['lsw']['LSWFINAL']==lsw_nr]
    
    # bereken de intersection tussen de lsw's en de peilgebieden
    data['lsw_peilgebied'] = gpd.overlay(lsw_data,
                                      data['peilgebieden'],
                                      how='intersection')
    
    data['lsw_peilgebied_top10nl'] = gpd.overlay(data['lsw_peilgebied'],
                                                 data['top10nl_gekoppeld'],
                                                 how='intersection')
    
    # bereken de intersection tussen de lsw's en top10nl
    data['lsw_top10nl'] = gpd.overlay(lsw_data,
                                      data['top10nl_gekoppeld'],
                                      how='intersection')
    
    # dissolve top10nl op koppelcodes
    data['lsw_top10nl_dissolved'] = data['lsw_top10nl'].dissolve('mozart_koppeling', as_index=False)
       
    # selecteer de data van de LSW's in het gebied van HDSR op basis van ID's
    df_mozart_lsw = f_wa.selecteer_LSWs(df_mozart,
                             [lsw_nr],
                             schrijf_csv=False)
    
    # check dat er naar 1 lsw wordt gekeken
    assert len(df_mozart_lsw['LSWNR'].unique())==1
    
    # verkrijg watervraag en wateraanbod voor de lsw
    df_vraag, df_aanbod = f_wa.mozart_to_df(df_mozart_lsw, demand_columns, alloca_columns)
    
    # bereken oppervlakte van de landgebruiktypen voor de lsw
    df_areas_lsw = pd.DataFrame(index=data['lsw_top10nl_dissolved']['mozart_koppeling'],
                                data=data['lsw_top10nl_dissolved'].area.values)
    
    df_percentages_lsw = df_areas_lsw/df_areas_lsw.sum()
    
    # verkrijg de unieke CODE-id's van de peilgebieden
    peilgebieden_codes = data['lsw_peilgebied_top10nl']['CODE'].unique()
    
    # loop over elk peilgebied met een unieke CODE
    for code in peilgebieden_codes[:]:
        # print(code)
        
        top10nl_peilgebied = data['lsw_peilgebied_top10nl'][data['lsw_peilgebied_top10nl']['CODE']==code]
        
        # dissolve top10nl op koppelcodes
        top10nl_peilgebied_dissolved = top10nl_peilgebied.dissolve('mozart_koppeling', as_index=False)
    
        df_areas_peilgebied = pd.DataFrame(index=top10nl_peilgebied_dissolved['mozart_koppeling'],
                                           data=top10nl_peilgebied_dissolved.area.values)
    
        percentage_peilgbd_lsw = df_areas_peilgebied/df_areas_lsw
        
        # zet NaN-waarden om naar 0%
        percentage_peilgbd_lsw.fillna(0, inplace=True)
    
    
        # bereken de watervraag, -aanbod en -tekorten voor de functies waar mogelijk 
        df_peilgebied = f_wa.verdeel_watervraag_aanbod(df_vraag, df_aanbod, percentage_peilgbd_lsw)
    
        if code in dict_peilgebieden.keys():
            # print(f'Peilgebied {code} al in dictionary. De resultaten worden gemerged.')
            
            dict_peilgebieden[code] = dict_peilgebieden[code] + df_peilgebied
            
        else:
            dict_peilgebieden[code] = copy.deepcopy(df_peilgebied)
            
#%% exporteer reeksen naar csv
for key, data_dict in dict_peilgebieden.items():
    data_dict.to_csv(fr'..\reeksen\baseline\mozart_peilgebied_{key}_geschaald.csv')

4a: check de reeksen

In [ ]:
test_naam = r'd:\projecten\hydrologische_informatieproducten\werkmap\reeksen\baseline\mozart_peilgebied_PG2077_geschaald.csv'

df_test = pd.read_csv(test_naam,
                     parse_dates=True,
                     index_col=['TIMEEND'])

fig, ax = plt.subplots()
df_test['watervraag_landbouw'].plot(ax=ax)
df_test['wateraanbod_landbouw'].plot(ax=ax)
ax.legend()
ax.set_ylabel('Volume water per decade [$m^3$/decade]')
ax.set_title('PG2077')

#5: Bepaal factor voor toekomstscenario

In [ ]:
# definieer de scenarios
scenarios = ['REF2017', 'W2050']

# definieer de map waar de scenario-data staan
directory = r'd:\projecten\hydrologische_informatieproducten\werkmap\data\lhm_toekomst\\'

# zet scenario-data om naar dataframe
dict_scenarios = f_wa.basisprognoses_naar_dataframe(directory, scenarios)

# bereken de gesommeerde watervraag en het gesommeerde wateraanbod
dict_watervraag_aanbod = dict()
for key, df in dict_scenarios.items():
    dict_watervraag_aanbod[key] = f_wa.mozart_watervraag_wateraanbod(df)

# bereken de factor tussen REF2017 en W2050
factor_vraag = dict_watervraag_aanbod['W2050']['watervraag']/dict_watervraag_aanbod['REF2017']['watervraag']
factor_aanbod = dict_watervraag_aanbod['W2050']['wateraanbod']/dict_watervraag_aanbod['REF2017']['wateraanbod']



# 5b: test voor watervaag

In [ ]:
df_test_toekomst = df_test['watervraag_landbouw']*factor_vraag.mean()

fig, ax = plt.subplots()
df_test['watervraag_landbouw'].plot(ax=ax)
df_test_toekomst.plot(ax=ax)
ax.legend()


diff = df_test_toekomst-df_test['watervraag_landbouw']
fig, ax = plt.subplots()
diff.plot(ax=ax)


# 6: Monte Carlo